In [40]:
import re
import pandas as pd
from datetime import datetime 

fecha = datetime.today().strftime('%Y%m%d')
num_format = re.compile("^[\-]?[0-9]*\.?[0-9]+$")

In [41]:
sheet = ['Caracteriza Papa ', 'Caracteriza Arveja', 'Caracteriza Cebolla', 'Caracteriza Zanahoria']
colName = ['L1_0', 'VAR', 'ARV_1', 'ARV_2', 'ARV_3', 'REND', 'ARV_4', 'ARV_5', 'ARV_6', 'ARV_7', 'ARV_8', 'ARV_9', 'ARV_10' ]

In [42]:
dftodo = pd.read_excel("D:\AEPS 2.0 Boyaca\datos\BD_SISTEMAS_AGRÍCOLAS-SOCIOECONÓMICOS-CLIMÁTICO\Caracterización.xlsx", sheet_name=sheet, header=1, usecols='D:P', names=colName)


In [43]:
dftodo['Caracteriza Papa ']['CULT'] = 'Papa'
dftodo['Caracteriza Arveja']['CULT'] = 'Arveja'
dftodo['Caracteriza Cebolla']['CULT'] = 'Cebolla'
dftodo['Caracteriza Zanahoria']['CULT'] = 'Zanahoria'
completo = pd.DataFrame()

completo = completo.append(dftodo['Caracteriza Papa '], ignore_index=True)
completo = completo.append(dftodo['Caracteriza Arveja'], ignore_index=True)#problemas con una CC en lugar de codigo geo farmer
completo = completo.append(dftodo['Caracteriza Cebolla'], ignore_index=True)
completo = completo.append(dftodo['Caracteriza Zanahoria'], ignore_index=True)

completo['L1_0'] = completo['L1_0'].astype(str).apply( lambda x: x[0:9].upper()) #Se eliminan letras al final del codigo agricultor
completo['ARV_6_raw'] = completo['ARV_6']
completo['ARV_6'] = completo['ARV_6'].astype(str).apply(lambda x: x.replace( "_", ""))
completo['ARV_6'] = completo['ARV_6'].astype(str).apply(lambda x: x.replace( " ", ""))
municipios = {
    'MO':'MOTAVITA',
    'SA':'SAMACÁ',
    'SI':'SIACHOQUE',
    'SO':'SORACÁ',
    'TO':'TOCA',
    'TU':'TUNJA',
    'VE':'VENTAQUEMADA',
}

def determiMuni (x):
    return municipios.get(x[0:2], '')

completo['L1_3'] = completo['L1_0'].apply(determiMuni)

completo.to_csv('completo.csv', index=False)


In [44]:
#agregar consecutivos en los datos repetidos de la serie para diferenciar
def consecutive(x, serie):
    conse = 1
    for j in x:
        if conse != 1 :
            serie[j] = serie[j] + f'-{conse}'
            print(f'Dato modificado en el indice {j} con: ' + ext_id_event[j])
        conse += 1

In [45]:
def duplicado_consecutive(serie):
    duplic = serie[serie.duplicated(keep=False)] #Se verifica si hay duplicados en la serie

    if not duplic.empty :
        duplic = pd.DataFrame(duplic)

        duplic = duplic.groupby(duplic.columns.tolist()).apply(lambda x: x.index.tolist())
        
        duplic.apply(consecutive, args=(serie,))
    return serie

In [46]:
#far_production_events
# 'technical'	'ext_id'	'plot'	'form'

ext_id_plot = 'LOTE-' + completo['L1_0'] #Se genera codigo del lote con el codigo del agricultor

ext_id_event = 'EVENTO-CUL-' + fecha + '-' + completo['L1_0'] #Se genera los id externos para los eventos productivos

ext_id_event = duplicado_consecutive(ext_id_event) #Se agrega un consecutivo a los id duplicados para diferenciar los eventos

completo['event'] = ext_id_event #Se agregan los id de evento para futuras implementaciones

e = {'technical' : 1, 'ext_id' : ext_id_event, 'plot': ext_id_plot, 'form' : 3}

event = pd.DataFrame(e)

event.to_csv('far_production_events.csv', index=False)

Dato modificado en el indice 514 con: EVENTO-CUL-20220317-MO-CA-001-2
Dato modificado en el indice 386 con: EVENTO-CUL-20220317-MO-CA-002-2
Dato modificado en el indice 515 con: EVENTO-CUL-20220317-MO-CA-002-3
Dato modificado en el indice 556 con: EVENTO-CUL-20220317-MO-CA-002-4
Dato modificado en el indice 400 con: EVENTO-CUL-20220317-MO-CA-011-2
Dato modificado en el indice 557 con: EVENTO-CUL-20220317-MO-CA-011-3
Dato modificado en el indice 391 con: EVENTO-CUL-20220317-MO-CA-016-2
Dato modificado en el indice 517 con: EVENTO-CUL-20220317-MO-CA-016-3
Dato modificado en el indice 519 con: EVENTO-CUL-20220317-MO-CA-018-2
Dato modificado en el indice 381 con: EVENTO-CUL-20220317-MO-CA-019-2
Dato modificado en el indice 554 con: EVENTO-CUL-20220317-MO-CA-019-3
Dato modificado en el indice 393 con: EVENTO-CUL-20220317-MO-CA-020-2
Dato modificado en el indice 383 con: EVENTO-CUL-20220317-MO-CA-025-2
Dato modificado en el indice 555 con: EVENTO-CUL-20220317-MO-CA-025-3
Dato modificado en e

In [48]:

cul = pd.DataFrame({'event': completo['event'], 'raw_value': completo['CULT'], 'question': "CULT", 'fixed_value': completo['CULT'],'validated': 1})
var = pd.DataFrame({'event': completo['event'], 'raw_value': completo['VAR'], 'question': "VAR", 'fixed_value': completo['VAR'],'validated': 1})
arv1 = pd.DataFrame({'event' : completo['event'], 'raw_value' : completo['ARV_1'], 'question': 'ARV_1', 'fixed_value': completo['ARV_1'] , 'raw_units': 'metros', 'fixed_units': 'metros', 'validated' : 1})
arv2 = pd.DataFrame({'event' : completo['event'], 'raw_value' : completo['ARV_2'], 'question': 'ARV_2', 'fixed_value': completo['ARV_2'] , 'raw_units': 'metros', 'fixed_units': 'metros', 'validated' : 1})
arv3 = pd.DataFrame({'event' : completo['event'], 'raw_value' : completo['ARV_3'], 'question': 'ARV_3', 'fixed_value': completo['ARV_3'] , 'raw_units': '', 'fixed_units': '', 'validated' : 1})
arv4 = pd.DataFrame({'event' : completo['event'], 'raw_value' : completo['ARV_4'], 'question': 'ARV_4', 'fixed_value': completo['ARV_4'] , 'raw_units': '%', 'fixed_units': '%', 'validated' : 1})
arv5 = pd.DataFrame({'event' : completo['event'], 'raw_value' : completo['ARV_5'], 'question': 'ARV_5', 'fixed_value': completo['ARV_5'] , 'raw_units': '%', 'fixed_units': '%', 'validated' : 1})
arv6 = pd.DataFrame({'event': completo['event'], 'raw_value': completo['ARV_6_raw'], 'question': "ARV_6", 'fixed_value': completo['ARV_6'],'validated': 1})
arv7 = pd.DataFrame({'event': completo['event'], 'raw_value': completo['ARV_7'], 'question': "ARV_7", 'fixed_value': completo['ARV_7'],'validated': 1})
arv8 = pd.DataFrame({'event': completo['event'], 'raw_value': completo['ARV_8'], 'question': "ARV_8", 'fixed_value': completo['ARV_8'],'validated': 1})
arv9 = pd.DataFrame({'event': completo['event'], 'raw_value': completo['ARV_9'], 'question': "ARV_9", 'fixed_value': completo['ARV_9'],'validated': 1})
arv10 = pd.DataFrame({'event': completo['event'], 'raw_value': completo['ARV_10'], 'question': "ARV_10", 'fixed_value': completo['ARV_10'],'validated': 1})
rend = pd.DataFrame({'event' : completo['event'], 'raw_value' : completo['REND'], 'question': 'REND', 'fixed_value': completo['REND'] , 'raw_units': 'Ton / ha', 'fixed_units': 'Ton / ha', 'validated' : 1})

far_responses_text = pd.concat([cul, var, arv6, arv7, arv8, arv9, arv10])
far_responses_numeric = pd.concat([arv1, arv2, arv3, arv4, arv5, rend])

far_responses_text.to_csv('far_responses_text.csv', index=False)
far_responses_numeric.to_csv('far_responses_numeric.csv', index=False)
